In [2]:
import pandas as pd
import numpy as np
pd.set_option('display.max_columns', 50)
# pd.set_option('display.max_rows', 50)
import tabula
from pandasql import sqldf
import math
from scipy.stats import chi2
from scipy import stats
from matplotlib import pyplot as plt

In [3]:
pmn = pd.read_excel(r'C:\Users\edwar\Desktop\農藝\paper2\data collection\Function\PMN_808.xlsx')

In [8]:
# drop duplicates
pmn = pmn.drop_duplicates(['pathway']).reset_index().drop(['index'], axis = 1)

In [9]:
DNA_inside_w = pd.read_excel(r"C:\Users\edwar\Desktop\農藝\paper2\data collection\integrated_results_DNAinside.xlsx", sheet_name = 'weighted')
dna_dt = DNA_inside_w.iloc[:135]
key_genes = pd.read_excel(r"C:\Users\edwar\Desktop\農藝\paper2\sys_biology\cytoscape\final_DTgenes.xlsx")

In [10]:
dtgenes = list(dna_dt['gene'])

In [11]:
pmn['N.genes'].describe()

count    607.000000
mean      23.856672
std       29.260820
min        0.000000
25%        6.000000
50%       14.000000
75%       30.000000
max      228.000000
Name: N.genes, dtype: float64

In [24]:
# pmn pw
pmn_v1 = pmn[(pmn['N.genes'] >= 4)].reset_index().drop(['index'], axis = 1)
print('orginal no.of pw {}'.format(len(pmn)))
print('number of pw remained {}'.format(len(pmn_v1)))

orginal no.of pw 607
number of pw remained 510


In [25]:
# finding at least one gene in pmn pws
dna_dtv1 = dna_dt.copy()
dna_dtv1['gene'] =  dna_dtv1['gene'].apply(lambda x : x.lower().replace('g', 'G'))
data = dna_dtv1
pathways = []
for i in range(len(pmn_v1['pathway'])):
    pathway_genes = pmn_v1[pmn_v1['pathway'] == pmn_v1['pathway'][i]].iloc[:, 3:3 + pmn_v1['N.genes'][i]].iloc[0].values
    if len(data[data['gene'].isin(pathway_genes)]) > 0:
        pathways.append(pmn_v1['pathway'][i])
    
    if (i + 1) / 50 == 0:
        print(i + 1)

In [26]:
print('number of pathways that at least contain one dt genes {}'.format(len(pathways)))

number of pathways that at least contain one dt genes 75


In [27]:
# only withhold pw containing dtgenes.
pmn_v2 = pmn_v1[pmn_v1['pathway'].isin(pathways)].reset_index().drop(['index'], axis = 1)

In [28]:
gene_list_pw = []
for i in range(len(pmn_v2['pathway'])):
    genes = pmn_v2[pmn_v2['pathway'] == pmn_v2['pathway'][i]].iloc[:, 3:3 + pmn_v2['N.genes'][i]].iloc[0].values
    gene_list_pw = list(gene_list_pw) + list(genes)
    gene_list_pw = set(gene_list_pw)
    
    if (i + 1) % 50 == 0:
        print(i + 1)
ttl_genes_pw = len(gene_list_pw)
print('total unique genes involved in pathways which have at least one DTgenes:', ttl_genes_pw)

50
total unique genes involved in pathways which have at least one DTgenes: 1590


In [29]:
gene_list_dt = []
for i in range(len(pmn_v2['pathway'])):
    genes = pmn_v2[pmn_v2['pathway'] == pmn_v2['pathway'][i]].iloc[:, 3:3 + pmn_v2['N.genes'][i]].iloc[0].values
    dt_gene = data[data['gene'].isin(genes)].reset_index().drop(['index'], axis = 1)['gene']
    gene_list_dt = list(gene_list_dt) + list(dt_gene)
    gene_list_dt = set(gene_list_dt)
    
    if (i + 1) % 50 == 0:
        print(i + 1)

ttl_genes_dt = len(gene_list_dt)
print('total DTgenes in pathways which have at least one DTgenes: ', ttl_genes_dt)

50
total DTgenes in pathways which have at least one DTgenes:  49


In [30]:
from scipy.stats import hypergeom
from statsmodels.stats.multitest import multipletests
# from statsmodels import stats
p = []

for i in range(len(pmn_v2['pathway'])):
    ttl_genes_inpw = pmn_v2.iloc[i]['N.genes']
    # npa 為存在在該 pw的 DT genes
    npa = pd.DataFrame(pmn_v2.iloc[i].iloc[3:3 + pmn_v2.iloc[i]['N.genes']]).reset_index().drop(['index'], axis = 1)
    npa.columns = ['gene']
    ttl_genes_dtin = len(npa[npa['gene'].isin(gene_list_dt)])
    
    # ttl_genes_dtin: No.DTgenes involved with genes in targeted pathway
    # ttl_genes_pw: total unique genes in all targeted pathways which include at least one DTgenes
    # ttl_genes_inpw: total genes in targeted pathway
    # ttl_genes_dt: No.DTgenes
    
    x, L, S, M = ttl_genes_dtin, ttl_genes_pw, ttl_genes_inpw, ttl_genes_dt
    p_val = hypergeom.sf(x - 1, L, S, M)
    p.append(p_val)
    p_BH = multipletests(p, alpha = 0.05, method = 'fdr_bh')[1]
    p_Bonf = multipletests(p, alpha = 0.05, method = 'bonferroni')[1]
    
    if (i + 1) % 50 == 0:
        print(i + 1)

50


In [31]:
pmn_v3 = pmn_v2[['pathway', 'N.genes']]
pmn_v3['p'] = p
pmn_v3['p_BH'] = p_BH
pmn_v3['p_Bonf'] = p_Bonf

C:\Users\edwar\AppData\Local\Temp\ipykernel_15332\747485494.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pmn_v3['p'] = p
C:\Users\edwar\AppData\Local\Temp\ipykernel_15332\747485494.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pmn_v3['p_BH'] = p_BH
C:\Users\edwar\AppData\Local\Temp\ipykernel_15332\747485494.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentatio

In [32]:
pmn_v3[pmn_v3['p'] < 0.05].reset_index().drop(['index'], axis = 1)

,pathway,N.genes,p,p_BH,p_Bonf
0,glycolysis I (from glucose 6-phosphate),146,0.011148,0.082457,0.836116
1,glycolysis II (from fructose 6-phosphate),133,0.005782,0.082457,0.433681
2,glycolysis IV (plant cytosol),128,0.004375,0.082457,0.328098
3,Calvin-Benson-Bassham cycle,78,0.008448,0.082457,0.633581
4,sucrose biosynthesis I (from photosynthesis),95,0.007107,0.082457,0.533026
5,superpathway of L-asparagine biosynthesis,18,0.016207,0.101296,1.000000
6,uracil degradation I (reductive),7,0.017703,0.102131,1.000000
7,S-adenosyl-L-methionine cycle II,27,0.008290,0.082457,0.621728
8,GDP-L-galactose biosynthesis,10,0.035760,0.167625,1.000000
9,UDP-alpha-D-glucuronate biosynthesis (from UDP...,10,0.035760,0.167625,1.000000


In [22]:
# pmn_v3[pmn_v3['p'] < 0.05].reset_index().drop(['index'], axis = 1).to_excel(r'C:\Users\edwar\Desktop\農藝\paper2\sys_biology\pmn_pathways.xlsx', index = False)

In [33]:
dna_go_hyper = pd.read_excel(r"C:\Users\edwar\Desktop\農藝\paper2\sys_biology\DNA_hypergeom_results.xlsx")

In [37]:
dna_go_hyper_v1 = dna_go_hyper[dna_go_hyper['p'] < 0.05].reset_index().drop(['index'], axis = 1)

In [44]:
dna_go_hyper_v1[dna_go_hyper_v1['Pathway'].str.contains('photosynthesis')]

,GO_id,Pathway,N.genes,p,p_BH,p_Bonf
98,GO:0009765,"photosynthesis, light harvesting",44,0.000039,0.000594,0.017232
122,GO:0015979,photosynthesis,449,0.013298,0.043000,1.000000


In [46]:
dna_go_hyper_v1[dna_go_hyper_v1['p_Bonf'] < 0.05]

,GO_id,Pathway,N.genes,p,p_BH,p_Bonf
13,GO:0004332,fructose-bisphosphate aldolase activity,15,4.308532e-07,1.468215e-05,1.908680e-04
19,GO:0004478,methionine adenosyltransferase activity,12,1.664675e-05,3.072712e-04,7.374510e-03
20,GO:0004553,"hydrolase activity, hydrolyzing O-glycosyl com...",541,2.360019e-05,4.181954e-04,1.045488e-02
24,GO:0004601,peroxidase activity,236,5.059313e-10,3.735459e-08,2.241276e-07
27,GO:0004721,phosphoprotein phosphatase activity,47,5.060058e-05,7.230986e-04,2.241606e-02
29,GO:0004866,endopeptidase inhibitor activity,46,4.644852e-05,6.858898e-04,2.057669e-02
31,GO:0005200,structural constituent of cytoskeleton,72,7.012559e-07,2.218974e-05,3.106563e-04
32,GO:0005507,copper ion binding,529,1.021963e-04,1.316859e-03,4.527296e-02
33,GO:0005618,cell wall,1355,5.775767e-48,2.558665e-45,2.558665e-45
36,GO:0005774,vacuolar membrane,1127,7.307647e-06,1.541566e-04,3.237288e-03
